In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import tqdm
# Be able to import from src folder
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
print(PROJ_ROOT)
import sys
sys.path.append(os.path.join(PROJ_ROOT, 'src'))

import misc.notebookHelpers

%matplotlib inline
%config InlineBackend.figure_format = "retina"

plt.rcParams['figure.figsize'] = (10, 10)

/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation


In [3]:
data_path = os.path.join(PROJ_ROOT, 'data/interim/old/ed_2_ed_clips')

In [4]:
data_path

'/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips'

In [5]:
h5files = glob.glob(os.path.join(data_path, '*/*.h5'))
h5files

['/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BBJ00_1.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BBJ00_2.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BC5G4_1.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BC5G4_2.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BCKG8_1.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BCKG8_2.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BD8OC_1.h5',
 '/Users/torjushaukom/Documents/Studier/Masteroppgave/Strain_estimation/data/interim/old/ed_2_ed_clips/p1/I53BD8OC_2.h5',
 '/Users/torjushaukom/Do

Pick a random file and see if we can find a suitable point

In [ ]:
file_idx = np.random.randint(0, len(h5files))
h5file = h5files[file_idx]
h5file

In [ ]:
data = h5py.File(h5file)
list(data.keys())

In [6]:
video = data['tissue/data'][:]
fps = 1 / (data['tissue/times'][3] - data['tissue/times'][2])

NameError: name 'data' is not defined

In [7]:
pal_txt = open('../../pal.txt')
line = pal_txt.readline()[:-1]
pal = np.array([int(val) for val in line.split(',')])

In [ ]:
video = np.array([[[pal[int(video[i, j, k])] for k in range(video.shape[2])] for j in range(video.shape[1])] for i in range(video.shape[0])])

Check that we can view the mitral points and that they make sense

In [ ]:
misc.notebookHelpers.ultraSoundAnimation(video, fps=fps / 8)

In [ ]:
import skimage

In [ ]:
median_video = np.zeros_like(video)
kernel = np.ones((7, 7))
for i in range(video.shape[0]):
    median_video[i, :, :] = skimage.filters.median(video[i, :, :], kernel)

In [ ]:
misc.ultraSoundAnimation(median_video, fps=fps)

In [ ]:
thresholded_video = median_video.copy()
thresholded_video[thresholded_video < 50] = 0

In [ ]:
misc.ultraSoundAnimation(thresholded_video, fps=fps)

In [ ]:
gaussian_video = thresholded_video.copy().astype(np.float32) / 255.

In [ ]:
for i in range(video.shape[0]):
    gaussian_video[i, :, :] = skimage.filters.gaussian(gaussian_video[i, :, :], sigma=3)

In [ ]:
misc.ultraSoundAnimation(gaussian_video, fps=fps)

In [ ]:
sobel_video = gaussian_video.copy()
for i in range(sobel_video.shape[0]):
    sobel_video[i, : :] = skimage.filters.sobel(gaussian_video[i, :, :], 
                                                video[i, :, :] != 0)

In [ ]:
misc.ultraSoundAnimation(sobel_video, fps=fps)

Select the first frame, and search for points below the mitral points to track

In [ ]:
frame = video[0, :, :]
mitral_points = data['tissue/track_points'][:2, :]

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')

In [9]:
import skimage

In [ ]:
kernel = np.ones((7, 7))
median_frame = skimage.filters.median(frame, kernel)

In [ ]:
fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(10, 15))
ax[0].imshow(frame, cmap='Greys_r')
ax[1].imshow(median_frame, cmap='Greys_r')

In [ ]:
plt.hist(np.ravel(median_frame))

In [ ]:
thresholded = median_frame.copy()
thresholded[thresholded < 20] = 0

plt.imshow(thresholded, cmap='Greys_r')

In [ ]:
import skimage.segmentation as segm

In [ ]:
segm_frame = segm.slic(median_frame, 3)
plt.imshow(segm_frame)

In [ ]:
data['tissue/pixelsize'][:]

In [8]:
def cm_to_pixel(num_cm):
    pixel_size_cm = 0.0005 * 100
    return int(round(num_cm / pixel_size_cm))

In [ ]:
search_box_left = np.array([[mitral_points[0, 0] - cm_to_pixel(4, data, 0), mitral_points[0, 0] + cm_to_pixel(1, data, 0)],
                            [mitral_points[0, 1] + cm_to_pixel(2, data, 1), mitral_points[0, 1] + cm_to_pixel(5, data, 1)]]).astype('int')

In [ ]:
search_box_left

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')

rectangle = plt.Rectangle((search_box_left[0, 0], search_box_left[1, 0]), 
                           search_box_left[0, 1] - search_box_left[0, 0], 
                           search_box_left[1, 1] - search_box_left[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

In [ ]:
left_search_region = frame[search_box_left[1, 0]:search_box_left[1, 1],
                           search_box_left[0, 0]:search_box_left[0, 1]]

In [ ]:
plt.imshow(left_search_region, cmap='Greys_r')

In [ ]:
flattened_index = np.argmax(left_search_region)
segment_point_left = np.unravel_index(flattened_index, left_search_region.shape)
segment_point_left = np.array([segment_point_left[1] + search_box_left[0, 0], segment_point_left[0] + search_box_left[1, 0]])
segment_point_left

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
plt.scatter(segment_point_left[0], segment_point_left[1], color='red')
rectangle = plt.Rectangle((search_box_left[0, 0], search_box_left[1, 0]), 
                           search_box_left[0, 1] - search_box_left[0, 0], 
                           search_box_left[1, 1] - search_box_left[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

Filter the image first

In [ ]:
import skimage

In [ ]:
kernel = np.ones((3, 3)) / 9
mean_filtered_frame = skimage.filters.rank.mean(frame.astype('uint8'), kernel)
plt.imshow(mean_filtered_frame, cmap='Greys_r')

In [ ]:
left_search_region = mean_filtered_frame[search_box_left[1, 0]:search_box_left[1, 1],
                           search_box_left[0, 0]:search_box_left[0, 1]]


In [ ]:
flattened_index = np.argmax(left_search_region)
segment_point_left = np.unravel_index(flattened_index, left_search_region.shape)
segment_point_left = np.array([segment_point_left[1] + search_box_left[0, 0], segment_point_left[0] + search_box_left[1, 0]])
segment_point_left

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
plt.scatter(segment_point_left[0], segment_point_left[1], color='red')

In [ ]:
search_box_right = np.array([[mitral_points[1, 0] - cm_to_pixel(1, data, 0), mitral_points[1, 0] + cm_to_pixel(4, data, 0)],
                             [mitral_points[1, 1] + cm_to_pixel(2, data, 1), mitral_points[1, 1] + cm_to_pixel(5, data, 1)]]).astype('int')

In [ ]:
right_search_region = mean_filtered_frame[search_box_right[1, 0]:search_box_right[1, 1],
                           search_box_right[0, 0]:search_box_right[0, 1]]

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')

rectangle = plt.Rectangle((search_box_left[0, 0], search_box_left[1, 0]), 
                           search_box_left[0, 1] - search_box_left[0, 0], 
                           search_box_left[1, 1] - search_box_left[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

rectangle = plt.Rectangle((search_box_right[0, 0], search_box_right[1, 0]), 
                           search_box_right[0, 1] - search_box_right[0, 0], 
                           search_box_right[1, 1] - search_box_right[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

In [ ]:
mask = frame != 0

In [ ]:
sobel_frame = skimage.filters.sobel(frame.astype('uint8'), mask=mask)
plt.imshow(sobel_frame, cmap='Greys_r')

In [ ]:
kernel = np.ones((3, 3)) / 9
mean_sobel = skimage.filters.median(sobel_frame)#, kernel)
plt.imshow(mean_sobel, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')

rectangle = plt.Rectangle((search_box_left[0, 0], search_box_left[1, 0]), 
                           search_box_left[0, 1] - search_box_left[0, 0], 
                           search_box_left[1, 1] - search_box_left[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

rectangle = plt.Rectangle((search_box_right[0, 0], search_box_right[1, 0]), 
                           search_box_right[0, 1] - search_box_right[0, 0], 
                           search_box_right[1, 1] - search_box_right[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

In [ ]:
plt.hist(np.ravel(mean_sobel))

In [ ]:
left_search_region = mean_sobel[search_box_left[1, 0]:search_box_left[1, 1],
                           search_box_left[0, 0]:search_box_left[0, 1]]

In [ ]:
right_search_region = mean_sobel[search_box_right[1, 0]:search_box_right[1, 1],
                           search_box_right[0, 0]:search_box_right[0, 1]]

In [ ]:
flattened_index = np.argmax(left_search_region)
segment_point_left = np.unravel_index(flattened_index, left_search_region.shape)
segment_point_left = np.array([segment_point_left[1] + search_box_left[0, 0], segment_point_left[0] + search_box_left[1, 0]])
segment_point_left

In [ ]:
flattened_index = np.argmax(right_search_region)
segment_point_right = np.unravel_index(flattened_index, right_search_region.shape)
segment_point_right = np.array([segment_point_right[1] + search_box_right[0, 0], segment_point_right[0] + search_box_right[1, 0]])
segment_point_right

In [ ]:
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
plt.scatter(segment_point_left[0], segment_point_left[1], color='red')
rectangle = plt.Rectangle((search_box_left[0, 0], search_box_left[1, 0]), 
                           search_box_left[0, 1] - search_box_left[0, 0], 
                           search_box_left[1, 1] - search_box_left[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

plt.scatter(segment_point_right[0], segment_point_right[1], color='red')
rectangle = plt.Rectangle((search_box_right[0, 0], search_box_right[1, 0]), 
                           search_box_right[0, 1] - search_box_right[0, 0], 
                           search_box_right[1, 1] - search_box_right[1, 0], color='r', fill=False)
plt.gca().add_patch(rectangle)

In [ ]:
mean_sobel[segment_point_left[1], segment_point_left[0]]

In [ ]:
mean_sobel[segment_point_right[1], segment_point_right[0]]

In [10]:
def triangleSector(top, height, left_theta, right_theta):
    left = [top[0] - height * np.tan(left_theta), top[1] + height]
    right = [top[0] + height * np.tan(right_theta), top[1] + height]
    left_barrier = []
    right_barrier = []
    for y in range(top[1], top[1] + height):
        x_left = int(np.round(((y - top[1]) / (left[1] - top[1])) * (left[0] - top[0]) + top[0]))
        left_barrier.append([x_left, y])
        
        x_right = int(np.round(((y - top[1]) / (right[1] - top[1])) * (right[0] - top[0]) + top[0]))
        right_barrier.append([x_right, y])
    left_barrier = np.array(left_barrier)
    right_barrier = np.array(right_barrier)
    return left_barrier, right_barrier

In [11]:
left_barrier, right_barrier = triangleSector(mitral_points[0, :], cm_to_pixel(4.5, data, 1), np.pi / 10, np.pi / 6)

NameError: name 'mitral_points' is not defined

In [ ]:
left_barrier = np.array(left_barrier)
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.plot(left_barrier[:, 0], left_barrier[:, 1])
plt.plot(right_barrier[:, 0], right_barrier[:, 1])

In [12]:
def searchTriangle(frame, top, height, left_theta, right_theta):
    left_barrier, right_barrier = triangleSector(top, height, left_theta, right_theta)
    max_val = 0
    max_idx = [0, 0]
    for i, y in enumerate(range(left_barrier[0, 1], left_barrier[-1, 1])):
        for j, x in enumerate(range(left_barrier[i, 0], right_barrier[i, 0] + 1)):
            if y >= left_barrier[0, 1] + cm_to_pixel(2) and x < frame.shape[1] and x >= 0:
                if frame[y, x] > max_val:
                    max_val = frame[y, x]
                    max_idx = [x, y]
    return max_idx, left_barrier, right_barrier

In [ ]:
left_point = searchTriangle(mean_sobel, mitral_points[0, :], cm_to_pixel(3.5, data, 1), np.pi / 8, np.pi / 8)
right_point = searchTriangle(mean_sobel, mitral_points[1, :], cm_to_pixel(3.5, data, 1), np.pi / 8, np.pi / 8)
plt.imshow(frame, cmap='Greys_r')
plt.scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
plt.scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
plt.scatter(left_point[0], left_point[1])
plt.scatter(right_point[0], right_point[1])

In [ ]:
data.close()

In [20]:
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [14]:
for fidx, h5file in enumerate(tqdm.tqdm_notebook(h5files)):
    file_name = h5file.split('/')[-1][:-3]
    data = h5py.File(h5file, 'r+')
    video = data['tissue/data'][:]
    fps = 1 / (data['tissue/times'][3] - data['tissue/times'][2])
    
    frame = video[0, :, :]
    frame = np.array([[pal[int(frame[j, k])] for k in range(video.shape[2])] for j in range(video.shape[1])])
    frame = frame / 255.
    mitral_points = data['tissue/track_points'][:2, :]

    x_dist = mitral_points[1, 0] - mitral_points[0, 0]
    y_dist = mitral_points[1, 1] - mitral_points[0, 1]
    
    mp_angle = np.arctan(y_dist / x_dist)
    # kernel = np.ones((7, 7)) / 49
    # mean_frame = skimage.filters.rank.mean(frame, kernel)
    # mask = frame != 0
    # sobel_frame = skimage.filters.sobel(frame, mask=mask)
    # mean_sobel = skimage.filters.rank.mean(sobel_frame, kernel)
    
    kernel = np.ones((7, 7))
    median_frame = skimage.filters.median(frame, kernel)
    thresholded = median_frame
    thresholded[thresholded < 20] = 0
    gaussian = skimage.filters.gaussian(thresholded, sigma=3)
    sobel_frame = skimage.filters.sobel(gaussian, frame != 0)
    
    if frame[mitral_points[0, 1], mitral_points[0, 0]] != 0.:
        left_point, left_lb, left_rb = searchTriangle(gaussian, 
                                                      mitral_points[0, :], 
                                                      cm_to_pixel(3.5), 
                                                      np.pi / 7 + mp_angle, 
                                                      0.1 - mp_angle)
    else:
        left_point = [0, 0]
        left_lb = np.zeros((15, 2))
        left_rb = np.zeros((15, 2))
    if frame[mitral_points[1, 1], mitral_points[1, 0]] != 0.:
        right_point, right_lb, right_rb = searchTriangle(gaussian, 
                                                         mitral_points[1, :], 
                                                         cm_to_pixel(3.5), 
                                                         0.1 + mp_angle, 
                                                         np.pi / 7 - mp_angle)
    else:
        right_point = [280, 0]
        right_lb = np.zeros((15, 2))
        right_rb = np.zeros((15, 2))
        
    track_points = np.vstack((mitral_points, left_point, right_point))
    data.create_dataset('tissue/det_track_points', data=track_points)
    # fig, ax = plt.subplots(ncols=4, sharex=True, sharey=True, figsize=(20, 5))
    # ax[0].imshow(frame, cmap='Greys_r')
    # ax[0].scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
    # ax[0].scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
    # ax[0].set_title('Original frame', fontsize=24)
    # ax[1].imshow(thresholded, cmap='Greys_r')
    # ax[1].set_title('Median filter and thresholding', fontsize=24)
    # ax[2].imshow(gaussian, cmap='Greys_r')
    # ax[2].set_title('Gaussian filter', fontsize=24)
    # ax[3].imshow(sobel_frame, cmap='Greys_r')
    # ax[3].plot(left_lb[:, 0], left_lb[:, 1], color='red')
    # ax[3].plot(left_rb[:, 0], left_rb[:, 1], color='red')
    # ax[3].plot(right_lb[:, 0], right_lb[:, 1], color='red')
    # ax[3].plot(right_rb[:, 0], right_rb[:, 1], color='red')
    # # ax[3].scatter(left_point[0], left_point[1], color='red')
    # # ax[3].scatter(right_point[0], right_point[1], color='red')
    # ax[3].set_title('Sobel filter and search sector', fontsize=24)
    # 
    # for axis in ax:
    #     axis.axis('off')
    # plt.tight_layout()
    # plt.savefig('../track_point_det.eps', dpi=150)
    # plt.close()
    # break
    fig, ax = plt.subplots(ncols=2, sharex=True, sharey=True, figsize=(10, 15))
    ax[0].imshow(frame, cmap='Greys_r')
    ax[0].scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
    ax[0].scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
    ax[0].scatter(left_point[0], left_point[1])
    ax[0].scatter(right_point[0], right_point[1])
    ax[0].plot(left_lb[:, 0], left_lb[:, 1], color='red')
    ax[0].plot(left_rb[:, 0], left_rb[:, 1], color='red')
    ax[0].plot(right_lb[:, 0], right_lb[:, 1], color='red')
    ax[0].plot(right_rb[:, 0], right_rb[:, 1], color='red')
    
    
    ax[1].imshow(sobel_frame, cmap='Greys_r')
    ax[1].scatter(mitral_points[0, 0], mitral_points[0, 1], color='red')
    ax[1].scatter(mitral_points[1, 0], mitral_points[1, 1], color='red')
    ax[1].scatter(left_point[0], left_point[1], color='red')
    ax[1].scatter(right_point[0], right_point[1], color='red')
    ax[1].plot(left_lb[:, 0], left_lb[:, 1], color='red')
    ax[1].plot(left_rb[:, 0], left_rb[:, 1], color='red')
    ax[1].plot(right_lb[:, 0], right_lb[:, 1], color='red')
    ax[1].plot(right_rb[:, 0], right_rb[:, 1], color='red')
    
    plt.savefig('../output/figures/old/track_points/' + file_name + '.png')
    plt.close()
    data.close()

/anaconda3/envs/strain/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in long_scalars
  from ipykernel import kernelapp as app


RuntimeError: Unable to create link (name already exists)

In [12]:
data.close()

In [ ]:
mitral_points

In [ ]:
right_point

In [ ]:
left_point

In [ ]:
np.vstack((mitral_points, left_point, right_point))

J49BK428_2